# Football Transfer News NLP 90MIN

Data scraped from https://www.90min.com/categories/transfer

Time taken to scrape 250 pages of articles ~ 1 hour and 15 minutes

In [66]:
def get_title(article):
    temp_list = article.find_all('h3')
    temp_list += article.find_all('h2')
    try:
        title = temp_list[0].text
    except:
        return None
    
    return title

def get_published_date(article):
    try:
        date = article.find('time').text
    except:
        return None
    return date

def get_article_link(article):
    try:
        link = article.find_all('a')[0].get('href', None)
    except:
        return None
    return link


def get_article_content(link):
    url = link
    result = requests.get(url).text
    article_doc = BeautifulSoup(result, 'html.parser')
    try:
        paragraphs = article_doc.find('div', {'class': 'wrapper_wfxu5h'}).find_all('p')
    except:
        return None
    container = []

    for paragraph in paragraphs[:-2]:
        container.append(paragraph.text)

    return " ".join(container)

In [67]:
def transferNews90minPageScraping(page_range):

    article_objects = []
    for page in tqdm.tqdm(page_range , total = len(page_range)):
        url = f'https://www.90min.com/categories/transfer?mm-experiments=finite-vs-recirculation%7Ctest&page={page}'
        result = requests.get(url).text
        doc = BeautifulSoup(result, 'html.parser')
        articles = doc.find_all('article')
        
        for article in articles:
            title = get_title(article)
            date = get_published_date(article)
            article_link = get_article_link(article)
            content = get_article_content(article_link)

            article_object = {
                'Title': title,
                'Date': date,
                'Link': article_link,
                'Content': content,
            }

            article_objects.append(article_object)

    return article_objects

In [68]:
from bs4 import BeautifulSoup
import requests
import tqdm
import pandas as pd

# page1_10 = transferNews90minPageScraping(range(1,11))

# page10_50 = transferNews90minPageScraping(range(10,51)) 

# page51_150 = transferNews90minPageScraping(range(51,151))

page151_250 = transferNews90minPageScraping(range(151,251))

100%|██████████| 100/100 [26:31<00:00, 15.92s/it]


In [69]:
import pandas as pd

pd.DataFrame(page151_250).to_csv('90minTransferNews_page151_250.csv',index=False)

In [70]:
df_1 = pd.DataFrame(page1_10)
df_2 = pd.DataFrame(page10_50).iloc[27:]
df_3 = pd.DataFrame(page51_150)
df_4 = pd.DataFrame(page151_250)

In [75]:
concat_df = pd.concat([df_1,df_2,df_3,df_4])
concat_df.to_csv("90minTransferNewsNLP.csv",index=False)

In [89]:
concat_df = concat_df.reset_index().drop(['level_0','index'],axis=1)

In [96]:
concat_df.iloc[0,1] = 'Aug 15, 2023'

In [98]:
concat_df.to_csv("90minTransferNewsNLP.csv", index=False)